I read a vert interesting blogs, [How we can make machine learning algorithm tunable](https://engraved.ghost.io/how-we-can-make-machine-learning-algorithms-tunable/), post Engraved by J. Degrave and I. Korshunova. 
Here, I summary the result and give my understanding.